In [50]:
import numpy as np
from mindspore.dataset import MnistDataset, GeneratorDataset
import pandas as pd
from mindspore import Tensor
import mindspore
from mindspore import nn, ops
from mindspore import dtype as mstype
from mindspore.dataset import transforms

In [51]:
data = pd.read_csv("D:\\Jupyter notebook\\data\\python\\iris_dataset\\irisInputs.csv",header=None)
label = pd.read_csv("D:\\Jupyter notebook\\data\\python\\iris_dataset\\irisTargets.csv",header=None)
data = data.values
label = label.values
type(label)

numpy.ndarray

In [52]:
def generator_func(data,label):
    """定义生成数据集函数"""
    for i in range(len(data)):
        yield (data[i],label[i])

In [53]:
dataset = GeneratorDataset(source=generator_func(data,label), column_names=["data", "label"])

[WARNING] ME(13700:15752,MainProcess):2022-11-03-11:45:39.696.676 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.


In [54]:
dataset = dataset.batch(batch_size=50)

In [49]:
for data,label in dataset.create_dict_iterator():
    print(data,label)

[WARNING] ME(13700:15752,MainProcess):2022-11-03-11:45:20.165.047 [mindspore\dataset\engine\iterators.py:143] No records available.


In [55]:
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(4, 4),
            nn.ReLU(),
            nn.Dense(4, 3)
        )

    def construct(self, x):
        logits = self.dense_relu_sequential(x)
        return logits

In [56]:
model = Network()
print(model)

Network<
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=4, output_channels=4, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=4, output_channels=3, has_bias=True>
    >
  >


In [57]:
def train_loop(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset):
        loss = train_step(data, label)
        loss, current = loss.asnumpy(), batch
        print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [58]:
learning_rate = 1e-2
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), learning_rate=learning_rate)

In [59]:
train_loop(model, dataset, loss_fn, optimizer)

[WARNING] ME(13700:15752,MainProcess):2022-11-03-11:45:46.242.412 [mindspore\dataset\engine\iterators.py:143] No records available.
